In [14]:
import torch
import numpy as np

from skimage.io import imread, imsave
from tqdm.auto import trange, tqdm
from torchvision.datasets import MNIST
from torchvision.transforms import Compose, Resize
from pytorch_fid import fid_score

from data_generator import DataGenerator
from default_mnist_config import create_default_mnist_config
from diffusion import DiffusionRunner
from models.classifier import ResNet, ResidualBlock, ConditionalResNet

from matplotlib import pyplot as plt

import os

#os.environ['CUDA_VISIBLE_DEVICES'] = '1'

#### Определим папку с настоящими картинками

In [10]:
def create_dir(path: str):
    if not os.path.exists(path):
        os.makedirs(path)

In [15]:
create_dir('../real_images_MNIST')

real_dataset = MNIST(root='../data', download=True, train=True, transform=Compose([Resize((32, 32))]))
for idx, (image_mnist, label) in enumerate(tqdm(real_dataset, total=len(real_dataset))):
    image = np.array(image_mnist)
    imsave("../real_images_MNIST/{}.png".format(idx), image)

100%|██████████| 9912422/9912422 [00:00<00:00, 221323988.52it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 99129045.68it/s]


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 67444475.40it/s]


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 17178114.31it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/60000 [00:00<?, ?it/s]

#### Определим папку для синтетических картинок и сгенерируем 60к картинок

In [19]:
device = torch.device('cuda')

In [16]:
uncond_diff = DiffusionRunner(create_default_mnist_config(), eval=True)

In [17]:
create_dir('../uncond_mnist')

TOTAL_IMAGES_COUNT = 10_000
BATCH_SIZE = 200
NUM_ITERS = TOTAL_IMAGES_COUNT // BATCH_SIZE

global_idx = 0
for idx in trange(NUM_ITERS):
    images: torch.Tensor = uncond_diff.sample_images(batch_size=BATCH_SIZE).cpu()
    images = images.permute(0, 2, 3, 1).data.numpy().astype(np.uint8)

    for i in range(len(images)):
        imsave(os.path.join('../uncond_mnist', f'{global_idx}.png'), images[i])
        global_idx += 1

  0%|          | 0/50 [00:00<?, ?it/s]

<ipython-input-17-0f2b55afcc2e>:13: UserWarning: ../uncond_mnist/465.png is a low contrast image
  imsave(os.path.join('../uncond_mnist', f'{global_idx}.png'), images[i])
<ipython-input-17-0f2b55afcc2e>:13: UserWarning: ../uncond_mnist/3846.png is a low contrast image
  imsave(os.path.join('../uncond_mnist', f'{global_idx}.png'), images[i])
<ipython-input-17-0f2b55afcc2e>:13: UserWarning: ../uncond_mnist/3888.png is a low contrast image
  imsave(os.path.join('../uncond_mnist', f'{global_idx}.png'), images[i])
<ipython-input-17-0f2b55afcc2e>:13: UserWarning: ../uncond_mnist/3935.png is a low contrast image
  imsave(os.path.join('../uncond_mnist', f'{global_idx}.png'), images[i])
<ipython-input-17-0f2b55afcc2e>:13: UserWarning: ../uncond_mnist/4637.png is a low contrast image
  imsave(os.path.join('../uncond_mnist', f'{global_idx}.png'), images[i])
<ipython-input-17-0f2b55afcc2e>:13: UserWarning: ../uncond_mnist/7275.png is a low contrast image
  imsave(os.path.join('../uncond_mnist', f'

In [21]:
fid_value = fid_score.calculate_fid_given_paths(
    paths=['../real_images_MNIST', '../uncond_mnist'],
    batch_size=200,
    device=device,
    dims=2048
)
fid_value

Downloading: "https://github.com/mseitzer/pytorch-fid/releases/download/fid_weights/pt_inception-2015-12-05-6726825d.pth" to /root/.cache/torch/hub/checkpoints/pt_inception-2015-12-05-6726825d.pth
100%|██████████| 91.2M/91.2M [00:01<00:00, 49.5MB/s]
100%|██████████| 50/50 [00:40<00:00,  1.23it/s]


118.36873256200087

> Какой фид получился? Сравните FID для безусловной генерации и для условной. Сгенерируйте для каждого класса по 6к картинок и посчитайте FID между реальными и условно сгенерированными картинками.

In [22]:
classifier_args = {
    "block": ResidualBlock,
    "layers": [2, 2, 2, 2]
}
noisy_classifier = ConditionalResNet(**classifier_args)
noisy_classifier.to(device)

noisy_classifier.load_state_dict(torch.load('./ddpm_checkpoints/classifier.pth'))

<All keys matched successfully>

In [23]:
conditional_diffusion = DiffusionRunner(create_default_mnist_config(), eval=True)
conditional_diffusion.set_classifier(noisy_classifier, T=1.)

In [28]:
create_dir('../cond_mnist')

TOTAL_IMAGES_COUNT = 10_000
BATCH_SIZE = 200
NUM_ITERS = TOTAL_IMAGES_COUNT // BATCH_SIZE

global_idx = 0
for idx in trange(NUM_ITERS):
    images: torch.Tensor = conditional_diffusion.sample_images(batch_size=BATCH_SIZE, labels=torch.tensor(list(range(10))*20).cuda()).cpu()
    images = images.permute(0, 2, 3, 1).data.numpy().astype(np.uint8)

    for i in range(len(images)):
        imsave(os.path.join('../cond_mnist', f'{global_idx}.png'), images[i])
        global_idx += 1

  0%|          | 0/50 [00:00<?, ?it/s]

<ipython-input-28-c101188675eb>:13: UserWarning: ../cond_mnist/1117.png is a low contrast image
  imsave(os.path.join('../cond_mnist', f'{global_idx}.png'), images[i])
<ipython-input-28-c101188675eb>:13: UserWarning: ../cond_mnist/2041.png is a low contrast image
  imsave(os.path.join('../cond_mnist', f'{global_idx}.png'), images[i])
<ipython-input-28-c101188675eb>:13: UserWarning: ../cond_mnist/2271.png is a low contrast image
  imsave(os.path.join('../cond_mnist', f'{global_idx}.png'), images[i])
<ipython-input-28-c101188675eb>:13: UserWarning: ../cond_mnist/6727.png is a low contrast image
  imsave(os.path.join('../cond_mnist', f'{global_idx}.png'), images[i])


In [29]:
fid_value = fid_score.calculate_fid_given_paths(
    paths=['../real_images_MNIST', '../cond_mnist'],
    batch_size=200,
    device=device,
    dims=2048
)
fid_value

100%|██████████| 51/51 [00:41<00:00,  1.23it/s]


120.25772186181388

FID для различных генераций получились примерно одинаковые